In [1]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
import time

from random import uniform, seed
seed(1)

from bs4 import BeautifulSoup

In [17]:
isin_1 = pd.read_csv('data/isin/isin_1.csv')
isin_2 = pd.read_csv('data/isin/isin_2.csv',index_col=0)
isin_34= pd.read_csv('data/isin/isin_34.csv',index_col=0)
ticker_isin = pd.read_csv('data/isin/ticker_isin.csv',index_col=0)

isin_1.columns = ['ticker', 'isin']
ticker_isin.columns = ['isin','ticker']

# isin이 us가 아닌 것
no_isin = pd.read_csv('data/isin/no_isin.csv',index_col=0)

no_isin=no_isin.reset_index().drop('index',axis=1)

# isin_new_2 
isin_new_2 = pd.read_csv('data/isin/isin_new_2.csv')

isin_new_2.columns =['ticker','isin']

# isin_us_6136 
isin_us_6136 = pd.read_csv('data/isin/isin_us_6136.csv',index_col=0)

# isin_us_6511
isin_us_6511 = pd.read_csv('data/isin/isin_us_6511.csv',index_col=0)

In [4]:
# isin_all = pd.concat([isin_1, isin_2, isin_34, ticker_isin], axis=0, ignore_index=True)
# isin_all=isin_all[isin_all['ticker'].notnull()]
# isin_all = isin_all.drop_duplicates(['ticker'], keep='first').reset_index().drop('index', axis=1)
# no_isin = isin_all[isin_all['isin'].str[:2]!="US"]

In [73]:
no_isin_2 = set(no_isin.ticker) - set(isin_us_6136.ticker)

### isin_crawling code

In [76]:
def isin_print(ticker):
    # ticker를 입력값으로 하여 이동할 url을 찾는다
    url= 'https://api.investing.com/api/search/v2/search?'
    data={
        "q": f"{ticker}"
    }
    header ={
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'origin': 'https://kr.investing.com',
    'referer': 'https://kr.investing.com/search/?q=950130',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    }
    response_url = requests.get(url,headers=header,params=data)
    
    # url 추출
    for i in response_url.json()['quotes']:
        if i['flag']=='USA':
            detail_url = i['url']
    
    # 찾은 url로 request한다.
    url_2='https://investing.com'+detail_url
    header_2 ={
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    }
    response = requests.get(url_2,headers=header_2)
    
    # request로 이동 후 BeautiSoup으로 element값을 통해 isin을 추출한다
    html_doc = response.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    news_tit = soup.findAll("span", {"class":"bottom-bar_value__1F-pW","data-test":"instrument-bottom-isin-value"})
    result = news_tit[0].text
    return result 

In [77]:
isin_0946 =  pd.Series()
for f_ticker in tqdm(no_isin_2):
    try:
        isin_0946[f'{f_ticker}'] = isin_print(f_ticker)
    except:
        isin_0946[f'{f_ticker}'] = 'no result'
        
    time.sleep(0.4)

C:\Users\gitple\AppData\Local\Temp\ipykernel_14608\2042252309.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  isin_0946 =  pd.Series()
 18%|██████████████▍                                                                | 426/2334 [08:51<33:04,  1.04s/it]C:\Anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
 54%|█████████████████████████████████████████▊                                    | 1251/2334 [26:24<18:22,  1.02s/it]C:\Anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
 62%|████████████████████████████████████████████████ 

In [78]:
isin_0946_csv = pd.DataFrame(isin_0946).reset_index()
isin_0946_csv.columns = ['ticker','isin']

isin_0946_csv_yes = isin_0946_csv[isin_0946_csv['isin'].str[:2]!='no']
isin_0946_csv_yes= isin_0946_csv_yes.reset_index().drop('index',axis=1)

In [107]:
isin_7254 = pd.concat([isin_us_6136,isin_0946_csv_yes],axis=0)
isin_7254 = isin_7254.reset_index().drop('index', axis=1)

In [109]:
isin_7254.to_csv('data/isin/isin_7254.csv')

### Check

In [34]:
ticker = 'BIPC'

In [35]:
url= 'https://api.investing.com/api/search/v2/search?'
data={
    "q": f"{ticker}"
}
header ={
'accept': 'application/json, text/javascript, */*; q=0.01',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
'origin': 'https://kr.investing.com',
'referer': 'https://kr.investing.com/search/?q=950130',
'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
}
response_url = requests.get(url,headers=header,params=data)

# url 추출
detail_url = response_url.json()['quotes'][0]['url']

# 찾은 url로 request한다.
url_2='https://investing.com'+detail_url
header_2 ={
'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
}
response = requests.get(url_2,headers=header_2)

# request로 이동 후 BeautiSoup으로 element값을 통해 isin을 추출한다
html_doc = response.text
soup = BeautifulSoup(html_doc, 'html.parser')
news_tit = soup.findAll("span", {"class":"bottom-bar_value__1F-pW","data-test":"instrument-bottom-isin-value"})
result = news_tit[0].text

In [46]:
# 1
for i in response_url.json()['quotes']:
    if i['flag']=='USA':
        detail_u = i['url']

In [54]:
# 2
detail_u=response_url.json()['quotes'][0]['url']

In [55]:
url_2='https://investing.com'+detail_u
header_2 ={
'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
}
response = requests.get(url_2,headers=header_2)

# request로 이동 후 BeautiSoup으로 element값을 통해 isin을 추출한다
html_doc = response.text
soup = BeautifulSoup(html_doc, 'html.parser')
news_tit = soup.findAll("span", {"class":"bottom-bar_value__1F-pW","data-test":"instrument-bottom-isin-value"})
result = news_tit[0].text

In [56]:
# 1
result

'CA11275Q1072'

In [48]:
# 2
result

'CA11275Q1072'

#### isin_check

In [36]:
# isin check
html_doc

'<!DOCTYPE html><html lang="en" dir="ltr" class="html"><head><link rel="dns-prefetch" href="//i-invdn-com.investing.com" /><link rel="dns-prefetch" href="//securepubads.g.doubleclick.net" /><link rel="dns-prefetch" href="//fonts.googleapis.com" /><link rel="preconnect" href="//i-invdn-com.investing.com" /><link rel="preconnect" href="//securepubads.g.doubleclick.net" /><link rel="preconnect" href="//live.primis.tech" /><link rel="preconnect" href="//fonts.googleapis.com" /><script>(function(){var __pm_s_v="83c155b";var pmJsonpEndPoint="CURRENT_JSONP_ENDPOINT";var pmJSHost="https://js.ad-score.com";var pid="1000745";var predefinedTags={tid:{qc:"utm_source",m:"refhost",v:""},l1:{qc:"utm_subsource",v:""},l2:{qc:"utm_medium",v:""},l3:{qc:"utm_campaign",v:""},l4:{qc:"utm_content",v:""},l5:window.location.hostname,l6:window.location.pathname};var runOnInnerPages=true;var predefinedTimeout=1500;var predefinedRequestsBeforeJSONP=3;var predefinedSampleRateRT=1;var predefinedSampleRateJS=\n1;var

### Making_Data

In [63]:
isin_2_clean = isin_new_2[isin_new_2['isin'].str[:2]!='no'].reset_index().drop('index',axis=1)

In [71]:
isin_2_clean=isin_2_clean[isin_2_clean['isin'].str[:2]!='US'].reset_index().drop('index',axis=1)

In [78]:
isin_us_6511 = pd.concat([isin_us_6136,isin_2_clean],axis=0,ignore_index=True)

In [83]:
isin_us_6511.to_csv('data/isin/isin_us_6511.csv')